In [5]:
import collections
import itertools
import json
import os
import re
import glob
import operator

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display
%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 20]

from IPython.display import display

In [64]:
vocab = set(json.load(open('../data/spider-20190205/nl2code-0214-nofrom-notinc/enc_vocab.json')))
left_preproc = [json.loads(line) for line in open('../data/spider-20190205/nl2code-0214-nofrom-notinc/enc/val.jsonl')]
def display_item(item):
    question_toks = left_preproc[item['i']]['question']
    
    IPython.display.display(IPython.display.HTML('''
    <ul>
        <li>Database: <tt>{db_id}</tt></li>
        <li>Question: {question_tok}</li>
        <li>Gold: <tt>{gold}</tt></li>
        <li>Left: <tt>{left}</tt></li>
        <li>Right: <tt>{right}</tt></li>
    </ul>
    '''.format(
        question_tok=' '.join((tok if tok in vocab else '<s>{}</s>'.format(tok)) for tok in question_toks),
        **item)))

In [38]:
def compare(data, left, right):
    both_exact = []
    left_exact = []
    right_exact = []
    neither_exact = []
    for i, (data_item, left_item, right_item) in enumerate(zip(data, left['per_item'], right['per_item'])):
        result = {
            'i': i,
            'db_id': data_item['db_id'],
            'question': data_item['question'],
            'gold': data_item['query'],
            'left': left_item['predicted'],
            'right': right_item['predicted'],
        }
        
        if left_item['exact'] and right_item['exact']:
            both_exact.append(result)
        elif left_item['exact']:
            left_exact.append(result)
        elif right_item['exact']:
            right_exact.append(result)
        else:
            neither_exact.append(result)
    return both_exact, left_exact, right_exact, neither_exact        

In [82]:
dev = json.load(open('../data/spider-20190205/dev.json'))
left = json.load(open('../logdirs/20190327/rerun,output_from=false,att=1/eval-val-step40000-bs1.jsonl'))
right = json.load(open('../logdirs/20190401/output_from=false,emb=glove-42B,min_freq=50,att=0/eval-val-step39100-bs1.jsonl')) 

In [84]:
both_exact, left_exact, right_exact, neither_exact = compare(dev, left, right)
total = len(dev)

In [86]:
len(both_exact) / total

0.17988394584139264

In [87]:
len(left_exact) / total

0.061895551257253385

In [88]:
len(right_exact) / total

0.19632495164410058

In [65]:
for item in both_exact[:100:5]:
    display_item(item)

In [66]:
for item in left_exact:
    display_item(item)

In [67]:
for item in right_exact[:100:5]:
    display_item(item)

In [89]:
for item in neither_exact[:100:5]:
    display_item(item)

In [80]:
both_exact, left_exact, right_exact, neither_exact = compare(
    dev,
    json.load(open('../logdirs/20190327/rerun,output_from=false,att=1/eval-val-step39100-bs1.jsonl')),
    json.load(open('../logdirs/20190327/rerun,output_from=false,att=1/eval-val-step40000-bs1.jsonl')))
len(both_exact), len(left_exact), len(right_exact)

(223, 13, 27)

In [78]:
both_exact, left_exact, right_exact, neither_exact = compare(
    dev,
    json.load(open('../logdirs/20190327/rerun,output_from=false,att=1/eval-val-step40000-bs1.jsonl')),
    json.load(open('../logdirs/20190327/rerun,output_from=false,att=3/eval-val-step38100-bs1.jsonl')))
len(both_exact), len(left_exact), len(right_exact)

(174, 76, 64)

In [76]:
both_exact, left_exact, right_exact, neither_exact = compare(
    dev,
    json.load(open('../logdirs/20190401/output_from=false,emb=bpemb-100k,min_freq=3,att=0/eval-val-step36100-bs1.jsonl')),
    json.load(open('../logdirs/20190401/output_from=false,emb=glove-42B,min_freq=50,att=0/eval-val-step39100-bs1.jsonl')))
len(both_exact), len(left_exact), len(right_exact)

(273, 102, 116)

In [81]:
both_exact, left_exact, right_exact, neither_exact = compare(
    dev,
    json.load(open('../logdirs/20190401/output_from=false,emb=glove-42B,min_freq=3,att=0/eval-val-step39100-bs1.jsonl')),
    json.load(open('../logdirs/20190401/output_from=false,emb=glove-42B,min_freq=50,att=0/eval-val-step39100-bs1.jsonl')))
len(both_exact), len(left_exact), len(right_exact)

(278, 66, 111)

In [90]:
# Fraction of count queries

In [91]:
dev[0]

{'db_id': 'concert_singer',
 'query': 'SELECT count(*) FROM singer',
 'query_toks': ['SELECT', 'count', '(', '*', ')', 'FROM', 'singer'],
 'query_toks_no_value': ['select', 'count', '(', '*', ')', 'from', 'singer'],
 'question': 'How many singers do we have?',
 'question_toks': ['How', 'many', 'singers', 'do', 'we', 'have', '?'],
 'sql': {'except': None,
  'from': {'conds': [], 'table_units': [['table_unit', 1]]},
  'groupBy': [],
  'having': [],
  'intersect': None,
  'limit': None,
  'orderBy': [],
  'select': [False, [[3, [0, [0, 0, False], None]]]],
  'union': None,
  'where': []}}